In [ ]:
# want to make a hypothetical backprograming for a aerial targeting program

In [ ]:
# need to make way better comments, probably lint

https://apps.dtic.mil/sti/pdfs/ADA522197.pdf

In [20]:
import numpy as np
import scipy
import time
import sched
import random

In [2]:
# constants for use in calculations

# rotation speed in search

# speed of projectile
missile_speed = 3.5 # m / s
# assume dragless, gravity negigible in short realm

In [3]:
def target_check(image):
  # test function to return true or false if a image
  # is recognized

  # to simulate, roll die with 1/10 probability
  if random.randint(1,10) > 9:
    return True
  else:
    return False

In [4]:
def rotate(phi_omega, theta_omega):
  # This function should call Ras Pi functions to rotate the camera or Weapon

  # insert raspi function for rotation in phi and theta directions
  return None

In [5]:
def capture_image():
  return image

In [ ]:
# This is the search mode

# This creates a 1 second delay for image search
# Assumes Camera and Weapon Angles are Same Until Mode Breaks

scheduler = sched.scheduler(time.time, time.sleep)
image = 'j'
search = True
target_spotted = False

target_mode = False
while search:

  scheduler.enter(1,1, print, ("SCANNING"))
  scheduler.run()

  if target_check("image"):
    search = False
    print("TARGET SPOTTED")
    target_mode = True
  else:
    #rotate(phi, theta)

In [ ]:
def search_mode():
  scheduler = sched.scheduler(time.time, time.sleep)
  image = 'j'
  search = True
  target_spotted = False

  target_mode = False
  while search:
    scheduler.enter(1,1, print, ("SCANNING"))
    scheduler.run()

    if target_check("image"):
      search = False
      print("TARGET SPOTTED")
      target_mode = True
    else:
      #rotate(phi, theta)

In [152]:
# to calculate the ballistics of a target, two samples is the minimum required for a triangle to be created

# simple example assumes a sonar image (probably want to generalize this to radar or image recognition)

class target_loc():
  def __init__(self, phi, theta, r, time_spot):
    self.phi = phi
    self.theta = theta
    self.r = r
    self.time_spot = time_spot
  def tell(self):
    return (f"Phi: {self.phi}  " + f"Theta: {self.theta} " + f"Rho: {self.r}")

In [9]:
def target_radar_sight(return_data):
  # need a function that captures a location of the target
  time = time.time()
  return phi, theta, r, time

In [120]:
# need to create a system for generating realistic trajectories of mock targets

#phi_one, theta_one, r_one, time_one = target_radar_sight(target_data_one)

#phi_two, theta_two, r_two, time_two = target_radar_sight(target_data_two)

first_loc = target_loc(.3, .3, 2, 0)

second_loc = target_loc(.301, 0.301, 2, 0.2)

In [13]:
def calculate_trajectory_target(first_data, second_data):
  # probably want to convert to cartesian coordinates immedatiately unless there is a spherical coordinate formula

  x_one = (first_data.r) * (np.cos(first_data.theta)) * (np.sin(first_data.phi))
  y_one = (first_data.r) * (np.sin(first_data.theta)) * (np.sin(first_data.phi))
  z_one = (first_data.r) * (np.cos(first_data.phi))

  x_two = (second_data.r) * (np.cos(second_data.theta)) * (np.sin(second_data.phi))
  y_two = (second_data.r) * (np.sin(second_data.theta)) * (np.sin(second_data.phi))
  z_two = (second_data.r) * (np.cos(second_data.phi))

  xyz_one = [x_one,y_one,y_two]
  xyz_two  = [x_two, y_two, z_two]

  deltaT = second_data.time_spot - first_data.time_spot

  deltaX = (x_two - x_one) / deltaT
  deltaY = (y_two - y_one) / deltaT
  deltaZ = (z_two - z_one) / deltaT

  deltaXYZ = [deltaX, deltaY, deltaZ]

  return [deltaT, deltaXYZ, xyz_one, xyz_two]

def recalculate_spherical(deltaX, deltaY, deltaZ, deltaT, x, y, z):

  dtheta_dt  = (-x / ((x ** 2) + (y ** 2)) ) * (deltaX) + (x / ((x ** 2) + (y ** 2)) ) * (deltaY)
  dphi_dt = ((z * x_two) / (sqrt(x ** 2 + y **2) * (x ** 2 + z **2 + y **2))) * (deltaX)  + ((z * x) / (sqrt(x ** 2 + y **2) * (x ** 2 + z **2 + y **2))) * (deltaY) - (sqrt(y ** 2 + x ** 2) / (z ** 2 + x **2 + y **2 )) * deltaZ
  dp_dt = (x / sqrt (x ** 2 + y ** 2 + z ** 2)) * deltaX + (y / sqrt(x ** 2 + y ** 2  + z **2)) * deltaY + (z / sqrt(x ** 2 + y ** 2  + z **2)) * deltaZ
  return [dtheta_dt, dphi_dt, dp_dt]

In [14]:
def calculate_ballistics_missile(speed, phi, theta):
  # probably want to convert to cartesian coordinates immedatiately unless there is a spherical coordinate formula

  deltaX = (speed) * (np.cos(theta)) * (np.sin(phi))
  deltaY = (speed) * (np.sin(theta)) * (np.sin(phi))
  deltaZ = (speed) * (np.cos(phi))

  deltaXYZ = [deltaX, deltaY, deltaZ]

  return deltaXYZ

In [167]:
def find_firing_solution(x, deltaXYZ, xyz_two, missile_speed):
  # x0 is time, x1 is theta, x2 is phi
  m_x_speed, m_y_speed, m_z_speed = calculate_ballistics_missile(missile_speed, x[2], x[1])
  t_x_speed, t_y_speed, t_z_speed = deltaXYZ
  t_x_pos, t_y_pos, t_z_pos = xyz_two
  return [0 + m_x_speed * x[0] - t_x_pos - (t_x_speed) * x[0], 0 + m_y_speed * x[0] - t_y_pos - (t_y_speed) * x[0], 0 + m_z_speed * x[0] - t_z_pos - (t_z_speed) * x[0]]

In [314]:
# code to generate random vector of target
# self, phi, theta, r, time_spot)
def generate_random_vector(max_distance):
  a = random.randrange(1,10, 1) * (np.pi / 2) / 10
  b = random.randrange(-10,10, 1) * (np.pi / 2) / 10

  data_one = target_loc(a, b, max_distance, 0)

  data_two = target_loc(a + 0.01, b + 0.01, max_distance + random.random(), 0.1)

  return [data_one, data_two]

In [285]:
first_loc, second_loc = generate_random_vector(2)

In [287]:
# probably going to have to create a differential equation solver for this one
deltaT, deltaXYZ, xyz_one, xyz_two = calculate_trajectory_target(first_loc, second_loc)

guess_solution = xyz_two
solution = xyz_two # need to replace this time factor with a better estimation

solution = scipy.optimize.fsolve(find_firing_solution, guess_solution, args=(deltaXYZ, xyz_two, missile_speed))

In [ ]:
if solution[0] <= .05:
  print("No SOLUTION FOUND")
else:
  print("SOLUTION CALCULATED:")
  print(f'TIME TO TARGET: {solution[0]}')
  print(f'Phi: {solution[1]}')
  print(f'Theta: {solution[2]}')

In [ ]:
# generate 10 solutions

for i in range(15):

  first_loc, second_loc = generate_random_vector(2)

  print("Target Azimuth : " + first_loc.tell())

  # probably going to have to create a differential equation solver for this one
  deltaT, deltaXYZ, xyz_one, xyz_two = calculate_trajectory_target(first_loc, second_loc)

  guess_solution = xyz_two
  solution = xyz_two # need to replace this time factor with a better estimation

  solution = scipy.optimize.fsolve(find_firing_solution, guess_solution, args=(deltaXYZ, xyz_two, missile_speed))
  solution
  if solution[0] <= .05:
    print("No SOLUTION FOUND")
    print("")
  else:
    print("SOLUTION CALCULATED:")
    print(f'TIME TO TARGET: {solution[0]}')
    print(f'Phi: {solution[1]}')
    print(f'Theta: {solution[2]}')
    print("")